### 4.4 기울기

편미분을 동시에 계산하고 싶을때는 어떻게 해야 할까?  
모든 변수의 편미분을 벡토로 정리한 것을 기울기(gradient)라고 한다.  

아래는 np.zeros_like() 함수를 통해 x와 형상이 같고 모두 0인 배열을 만들어서 계산된 기울기를 저장하는 코드이다.  
넘파이 배열 x의 각 원소에 대한 수치 미분을 구한다.  

In [4]:
import numpy as np

def function_2(x):
    return x[0]**2 + x[1]**2
    # return np.sum(x**2)

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성
    
    for idx in range(x.size):
        tmp_val = x[idx]
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 값 복원
        
    return grad

print(numerical_gradient(function_2, np.array([3.0, 4.0])))
print(numerical_gradient(function_2, np.array([0.0, 2.0])))
print(numerical_gradient(function_2, np.array([3.0, 0.0])))

[6. 8.]
[0. 4.]
[6. 0.]


이런 기울기가 의미하는 바가 무엇일까?  
기울기 결과에 마이너스를 붙인 벡터를 그려보면 방향을 가진 벡터로 표시된다.  
그리고 이는 가장 낮은 장소, 최솟값을 가르키고 있다. 낮은 곳에서 멀어질 수록 화살표의 크기가 커짐을 알 수 있다.  
정확히는 기울기가 가리키는 쪽은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향이다!!!  
기울기는 각 지점에서 낮아지는 방향을 가리킨다.  

#### 4.4.1 경사법(경사 하강법)

기울기를 이용해 함수의 최솟값을 찾는 것이 경사법이다.  
다만 각 지점에서의 기울기가 가리키는 곳에 정말 함수의 최솟값이 있는지 보장할 수는 없다. 최솟값이 없는 경우가 실제로는 대다수이다.  
함수가 극솟값, 최솟값, 안장점이 되는 장소에서는 기울기가 0이 된다.  

하지만 그 방향으로 가야 함수의 값을 줄일 수 있다. 그래서 기울기 정보를 단서로 나아갈 방향을 정해야 한다.  
여기서 경사법이 등장한다. 현 위치에서 기울어진 방향으로 일정 거리만큼 이동한다. 그 뒤 이동한 곳에서도 기울기를 구해 나아가기를 반복한다. 이렇게 함수의 값을 점차 줄이는 것이 경사법이다.  

구해진 기울기와 학습률(갱신하는 양)을 곱하여 원래 매개변수의 값에서 뺌으로 함수의 값을 줄여나간다. 변수가 여러개여도 각 변수의 편미분 값으로 갱신하게 된다. 이때 학습률은 0.01이나 0.001 등 미리 지정해주어야 한다. 너무 크거나 작으면 좋은 곳을 갈 수 가 없다.  

In [10]:
# f는 최적화하려는 함수, init_x는 초깃값, lr = 학습률, step_num = 기울기에 학습률을 곱한 값으로 갱신하는 처리를 반복하는 횟수

def gradient_descent(f, init_x, lr = 0.01, step_num = 100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x


# 초깃값 설정 후 최솟값 탐색을 시작한다.

def function_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])
print(gradient_descent(function_2, init_x = init_x, lr = 0.1, step_num = 100))

# 학습률이 너무 크면 큰 값으로 발산해버린다.
init_x = np.array([-3.0, 4.0])
print(gradient_descent(function_2, init_x = init_x, lr = 10.0, step_num = 100))

# 학습률이 너무 작으면 갱신되지 않는다.
init_x = np.array([-3.0, 4.0])
print(gradient_descent(function_2, init_x = init_x, lr = 1e-10, step_num = 100))

[-6.11110793e-10  8.14814391e-10]
[-2.58983747e+13 -1.29524862e+12]
[-2.99999994  3.99999992]


사람이 직접 설정해야 하는 매개변수를 하이퍼파라미터라 칭한다.  
이는 여러 시험을 통해 찾아가는 과정을 거친다.

#### 4.4.2 신경망에서의 기울기

신경망 학습시에도 기울기를 구해야 한다. 가중치 매개변수에 대한 손실 함수의 기울기이다.  
가중치가 W, 손실함수가 L일때 경사의 원소는 각각의 원소에 관한 편미분이다. 기울기의 형상은 W와 같다.  

In [16]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3) # 정규분포로 초기화
    
    def predict(self, w):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

SimpleNet 클래스는 형상이 2 X 3인 가중치 매개변수 하나를 인스턴스 변수로 갖는다.  
메서드는 예측하는 predict와 손실함수 값을 구하는 loss이다.  

In [17]:
net = simpleNet()
print(net.W)

x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

print(np.argmax(p))

t = np.array([0, 0, 1])
net.loss(x, t)

[[-0.27182878  0.77005924  0.51947118]
 [-1.8058318   0.30137102  0.54515341]]
[-1.78834589  0.73326946  0.80232078]
2


0.6972629113366094

In [18]:
def f(W):
    return net.loss(x, t)

dw = numerical_gradient(f, net.W)
print(dw)

[[ 0.02239863  0.27883355 -0.30123218]
 [ 0.03359794  0.41825033 -0.45184827]]


w11을 h만큼 늘리면 손실함수의 값은 0.2h 만큼 증가한다는 뜻이다.  
반면 w23은 대략 -0.45이니 h만큼 늘리면 0.45h 만큼 감소한다는 뜻이다.  
손실함수를 줄이려면 w11은 음의 방향으로, w23은 양의 방향으로 갱신해야 한다.  

In [19]:
f = lambda w: net.loss(x, t)
dw = numerical_gradient(f, net.W)